In [1]:
import os
import re
import cv2
import numpy as np
#from scipy import ndimage
import scipy.ndimage as nd
from twilio.rest import Client
from skimage.util import view_as_blocks
from multiprocessing import Pool

In [2]:
# Your Account SID from twilio.com/console
account_sid = "AC87af7fb47f157646e91bc4f5b9e87797"
# Your Auth Token from twilio.com/console
auth_token  = "be5f07b59a5c3b27c1b44fd0a7e6c9ca"

client = Client(account_sid, auth_token)


In [3]:
def natural_keys(text):
    '''
    alist.sort(key=natural_keys) sorts in human order
    http://nedbatchelder.com/blog/200712/human_sorting.html
    '''
    return [ atoi(c) for c in re.split('(\d+)', text) ]
def atoi(text):
    return int(text) if text.isdigit() else text
def GetID(name):
    '''
    FIXME! depend on filename, need to improve!
    '''
    k=name.split('.')[-2]
    return int(k[-13:-9])-1

In [4]:
def generic_laplace(input, derivative2, output=None, mode="reflect",
                    cval=0.0,
                    extra_arguments=(),
                    extra_keywords = None):
    """N-dimensional Laplace filter using a provided second derivative function
    Parameters
    ----------
    %(input)s
    derivative2 : callable
        Callable with the following signature::
            derivative2(input, axis, output, mode, cval,
                        *extra_arguments, **extra_keywords)
        See `extra_arguments`, `extra_keywords` below.
    %(output)s
    %(mode)s
    %(cval)s
    %(extra_keywords)s
    %(extra_arguments)s
    """
    if extra_keywords is None:
        extra_keywords = {}
    input = numpy.asarray(input)
    output, return_value = _ni_support._get_output(output, input)
    axes = list(range(input.ndim))
    if len(axes) > 0:
        derivative2(input, axes[0], output, mode, cval,
                    *extra_arguments, **extra_keywords)
        for ii in range(1, len(axes)):
            tmp = derivative2(input, axes[ii], output.dtype, mode, cval,
                              *extra_arguments, **extra_keywords)
            output += tmp
    else:
        output[...] = input[...]
    return return_value

def gaussian_laplace(input, sigma, output=None, mode="reflect",
                     cval=0.0, **kwargs):
    """Multidimensional Laplace filter using gaussian second derivatives.
    Parameters
    ----------
    %(input)s
    sigma : scalar or sequence of scalars
        The standard deviations of the Gaussian filter are given for
        each axis as a sequence, or as a single number, in which case
        it is equal for all axes.
    %(output)s
    %(mode)s
    %(cval)s
    Extra keyword arguments will be passed to gaussian_filter().
    """
    input = numpy.asarray(input)

    def derivative2(input, axis, output, mode, cval, sigma, **kwargs):
        order = [0] * input.ndim
        order[axis] = 2
        return gaussian_filter(input, sigma, order, output, mode, cval,
                               **kwargs)

    return generic_laplace(input, derivative2, output, mode, cval,
                           extra_arguments=(sigma,),
                           extra_keywords=kwargs)


def hist_match(source, template):
    """
    Adjust the pixel values of a grayscale image such that its histogram
    matches that of a target image

    Arguments:
    -----------
        source: np.ndarray
            Image to transform; the histogram is computed over the flattened
            array
        template: np.ndarray
            Template image; can have different dimensions to source
    Returns:
    -----------
        matched: np.ndarray
            The transformed output image
    """

    oldshape = source.shape
    source = source.ravel()
    template = template.ravel()

    # get the set of unique pixel values and their corresponding indices and
    # counts
    s_values, bin_idx, s_counts = np.unique(source, return_inverse=True,
                                            return_counts=True)
    t_values, t_counts = np.unique(template, return_counts=True)

    # take the cumsum of the counts and normalize by the number of pixels to
    # get the empirical cumulative distribution functions for the source and
    # template images (maps pixel value --> quantile)
    s_quantiles = np.cumsum(s_counts).astype(np.float64)
    s_quantiles /= s_quantiles[-1]
    t_quantiles = np.cumsum(t_counts).astype(np.float64)
    t_quantiles /= t_quantiles[-1]

    # interpolate linearly to find the pixel values in the template image
    # that correspond most closely to the quantiles in the source image
    interp_t_values = np.interp(s_quantiles, t_quantiles, t_values)

    return interp_t_values[bin_idx].reshape(oldshape)

def hist_match_color(source,template):
    b_source=source[:,:,0]
    g_source=source[:,:,1]
    r_source=source[:,:,2]
    
    b_template=template[:,:,0]
    g_template=template[:,:,1]
    r_template=template[:,:,2]
    
    b_matched=hist_match(b_source,b_template)
    g_matched=hist_match(g_source,g_template)
    r_matched=hist_match(r_source,r_template)
    
    matched=cv2.merge([b_matched,g_matched,r_matched])
    return matched
def info(img8):
    print img8.dtype
    print img8.shape
    print type(img8)
    print np.max(img8)
    print np.min(img8)
def Down_Sample(image, block_size, func=np.sum, cval=0):

    if len(block_size) != image.ndim:
        raise ValueError("`block_size` must have the same length "
                         "as `image.shape`.")

    pad_width = []
    for i in range(len(block_size)):
        if block_size[i] < 1:
            raise ValueError("Down-sampling factors must be >= 1. Use "
                             "`skimage.transform.resize` to up-sample an "
                             "image.")
        if image.shape[i] % block_size[i] != 0:
            after_width = block_size[i] - (image.shape[i] % block_size[i])
        else:
            after_width = 0
        pad_width.append((0, after_width))

    image = np.pad(image, pad_width=pad_width, mode='constant',
                   constant_values=cval)

    out = view_as_blocks(image, block_size)

    for i in range(len(out.shape) // 2):
        out = func(out, axis=-1)

    return out

In [5]:
def GetBlockList(overlap=5):
    x1=0
    y1=256
    blocklist=[]
    count = 0
    while (x1+256<10000):
        x2=0
        y2=256
        while (x2+256<12000):
            blocklist.append(np.asarray([x1,y1,x2,y2,count]))
            count = count + 1
            x2=y2-overlap
            y2=x2+256
        x1=y1-overlap
        y1=x1+256
    return blocklist

In [6]:
def blockprocess(img,metadata,filecount):
    #print type(metadata)
    img=np.asarray(img,'uint8')
    sub=img[metadata[0]:metadata[1],metadata[2]:metadata[3]]
    #Gmask= np.max(sub,axis=2) != sub[:,:,1]
    #sub[Gmask]=0
    #sub[sub[:,:,1]<50] =0
    #sub=cv2.cvtColor(sub,cv2.COLOR_BGR2GRAY)
    #cv2.imwrite(str(metadata[4])+'/'+str(filecount)+'.jp2',sub)
    #print '/scratch/PMD1463_SK/'+str(metadata[4])+'/'+str(filecount)+'.jp2'
    cv2.imwrite('/scratch/PMD1463_SK/'+str(metadata[4])+'/'+str(filecount)+'.jp2',sub)

In [7]:
def ProcessOne(filename):
    '''
    read the file by -1
    take[20000,24000]
    LOG
    filter
    gray
    8bit
    eq
    blur
    downsample
    cut
    write
    '''
    print 'processing '+filename
    img = cv2.imread(filename,-1)
    print img.shape
    img = img[0:20000,0:24000,:]
    LOG=nd.gaussian_laplace(img,3)

    mask1 = LOG[:,:,0] + LOG[:,:,1] + LOG[:,:,2] < 2000     #background noise 40
    mask2 = img[:,:,1] < 400 #np.mean(img[:,:,1])       #not strong green (need to replace by better detect) 70
    mask3 = np.max(img[:,:,:],axis=2) == img[:,:,2]   #looks red(need to replace by autofluo detect)


    img[mask1 & mask2] = 0 
    img[mask3 & mask2] = 0

    img=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    img=np.asarray(img/float(16),'uint8')

    img=cv2.equalizeHist(img)
    img=cv2.medianBlur(img,3)
    print img.shape
    img=Down_Sample(img,block_size=(2,2),func=np.mean)
    print img.shape
    blocks=GetBlockList()
    counts=GetID(filename)
    
    for k in blocks:
        blockprocess(img,k,counts)
    #print info(img)
    #cv2.imwrite('/scratch/PMD1463_NOCUT/'+os.path.basename(filename),img)
    return img
    #print 'finishing '+filename

In [8]:
inputfolder='/scratch/PMD1463/'
outputfolder='/scratch/PMD1463_SK/'

filelist=sorted(os.listdir(inputfolder),key=natural_keys)
filelist=[inputfolder+x for x in filelist]

In [ ]:
p=Pool(10)
p.map(ProcessOne, filelist)

In [ ]:
#for i in range(1832):
    #os.mkdir(outputfolder+str(i))

In [ ]:
message = client.messages.create(
    to="+12174192723", 
    from_="+12175744190",
    body="主人你的程序跑完啦！")

print(message.sid)

In [9]:
img=ProcessOne(filelist[0])

processing /scratch/PMD1463/PMD1464&1463-F1-2014.01.31-18.28.08_PMD1463_1_0001_lossless.jp2
(15861, 18906, 3)
(15861, 18906)
(7931, 9453)


In [ ]:
info(img)

In [10]:
block=GetBlockList()

In [13]:
for i in block:
    print str(i[0])+' '+str(i[2])+' '+str(0)

0 0 0
0 251 0
0 502 0
0 753 0
0 1004 0
0 1255 0
0 1506 0
0 1757 0
0 2008 0
0 2259 0
0 2510 0
0 2761 0
0 3012 0
0 3263 0
0 3514 0
0 3765 0
0 4016 0
0 4267 0
0 4518 0
0 4769 0
0 5020 0
0 5271 0
0 5522 0
0 5773 0
0 6024 0
0 6275 0
0 6526 0
0 6777 0
0 7028 0
0 7279 0
0 7530 0
0 7781 0
0 8032 0
0 8283 0
0 8534 0
0 8785 0
0 9036 0
0 9287 0
0 9538 0
0 9789 0
0 10040 0
0 10291 0
0 10542 0
0 10793 0
0 11044 0
0 11295 0
0 11546 0
251 0 0
251 251 0
251 502 0
251 753 0
251 1004 0
251 1255 0
251 1506 0
251 1757 0
251 2008 0
251 2259 0
251 2510 0
251 2761 0
251 3012 0
251 3263 0
251 3514 0
251 3765 0
251 4016 0
251 4267 0
251 4518 0
251 4769 0
251 5020 0
251 5271 0
251 5522 0
251 5773 0
251 6024 0
251 6275 0
251 6526 0
251 6777 0
251 7028 0
251 7279 0
251 7530 0
251 7781 0
251 8032 0
251 8283 0
251 8534 0
251 8785 0
251 9036 0
251 9287 0
251 9538 0
251 9789 0
251 10040 0
251 10291 0
251 10542 0
251 10793 0
251 11044 0
251 11295 0
251 11546 0
502 0 0
502 251 0
502 502 0
502 753 0
502 1004 0
502 1255 

In [14]:
block[-1]

array([ 9538,  9794, 11546, 11802,  1832])